In [ ]:
!pip install roboflow
!pip install ultralytics

In [4]:
# Импорт библиотек
from roboflow import Roboflow
from ultralytics import YOLO

In [ ]:
# Получение и подготовка датасета для обучения модели
rf = Roboflow(api_key="091mcn4Yvn3gWaa9FCeq")
project = rf.workspace("parkinglot-64itp").project("parking_lot-7nsmw")
version = project.version(19)
dataset = version.download("yolov8")

In [ ]:
# Загружаем предобученную модель для обучения 
model = YOLO("yolov8n.pt")

# Тренировка
model.train(data="Parking_Lot-19/data.yaml", epochs=20, imgsz=640)  # У меня ушло полтора часа

In [7]:
# Загрузка твоих весов
model = YOLO("runs/detect/train9/weights/best.pt")

# Тест на одной картинке
results = model.predict("5.png", save=True, imgsz=640, show_labels=False, show_conf=False)


image 1/1 C:\Users\1\_JupyterNotebook_content\_ParkingML\5.png: 640x576 34 emptylots, 8 nonemptylots, 113.7ms
Speed: 8.2ms preprocess, 113.7ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 576)
Results saved to runs\detect\predict9
